In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Menyiapkan semua library yang dibutuhkan

In [3]:
# import packages
import csv
import numpy as np
import pandas as pd
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, losses
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt




### Data Wrangling

#### Gathering Data

In [5]:
# import data
dataset1_df = pd.read_csv('/content/sample_data/dataset1.csv')
dataset1_df #show data below

,ingredient_name,rating,functions,link
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...
...,...,...,...,...
26082,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN
26083,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN
26084,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN
26085,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN


In [6]:
# import data
dataset2_df = pd.read_csv('/content/sample_data/dataset2.csv')
dataset2_df #show data below

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Evens Skin Tone, Dark Spot Fading",Antioxidant,A more stable derivative of pure vitamin C (as...,"Free Radical Biology and Medicine, September ...",INGREDIENT DICTIONARY/SEARCH RESULTS\n3-O Ethy...,"3-O ethyl ascorbic acid is a stable, water- an...","3-O ethyl ascorbic acid is a stable, water- an..."
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Soothing","Antioxidant, Plant Extracts",NaN,NaN,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcai\nRa...,Acai (pronounced “ah-sigh-ee”) is a small berr...,Acai (pronounced “ah-sigh-ee”) is a small berr...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,"Antioxidant, Plant Extracts",NaN,"Journal of AOAC International, September 2018...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcerola ...,Acerola fruit extract (also known as _Malpighi...,Acerola fruit extract (also known as _Malpighi...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...,Hydration,Antioxidant,NaN,Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl C...,NaN,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,Peptides,Lab-made anti-aging peptide that promotes firm...,"Pharmaceuticals, July 2021, pages 1–22;;Jour...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl D...,Acetyl dipeptide-1 cetyl ester is a lab-engine...,Acetyl dipeptide-1 cetyl ester is a lab-engine...
...,...,...,...,...,...,...,...,...,...,...,...
26069,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Skin Conditioning. We have not yet...
26070,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Antioxidant. We have not yet rated...
26071,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Humectant. We have not yet rated t...
26072,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Functions: Humectant, Skin Conditioning. We h..."


### Assesing Data

#### Menilai Tabel dataset1_df

In [7]:
dataset1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26087 entries, 0 to 26086
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26087 non-null  object
 1   rating           26087 non-null  object
 2   functions        26072 non-null  object
 3   link             2412 non-null   object
dtypes: object(4)
memory usage: 815.3+ KB


In [8]:
dataset1_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,15
link,23675


In [9]:
print("Jumlah Duplikasi: ", dataset1_df.duplicated().sum())

Jumlah Duplikasi:  0


In [10]:
dataset1_df.describe()

,ingredient_name,rating,functions,link
count,26087,26087,26072,2412
unique,26084,6,4055,2412
top,Albumen,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...
freq,2,23675,5642,1


#### Menilai Tabel dataset2_df

In [11]:
dataset2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26074 non-null  object
 1   rating           26074 non-null  object
 2   functions        26060 non-null  object
 3   link             2399 non-null   object
 4   benefits         1524 non-null   object
 5   categories       2350 non-null   object
 6   glance           942 non-null    object
 7   references       880 non-null    object
 8   all              2382 non-null   object
 9   description      2370 non-null   object
 10  combined_text    26073 non-null  object
dtypes: object(11)
memory usage: 2.2+ MB


In [12]:
dataset2_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,14
link,23675
benefits,24550
categories,23724
glance,25132
references,25194
all,23692
description,23704


In [13]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


In [14]:
dataset2_df.describe()

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
count,26074,26074,26060,2399,1524,2350,942,880,2382,2370,26073
unique,26074,6,4044,2399,90,392,927,367,2382,2350,4245
top,3-O Ethyl Ascorbic Acid,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...,Hydration,"Antioxidant, Plant Extracts","Plant extract with proven skin-soothing, antio...",Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\n3-O Ethy...,Used as an emollient and thickening agent in c...,Functions: Skin Conditioning. We have not yet...
freq,1,23675,5642,1,501,255,3,241,1,7,5642


### Cleaning Data

#### Cleaning Tabel dataset1_df

##### Duplicate Data

In [15]:
dataset1_df.duplicated().sum()

0

In [16]:
dataset1_df.drop_duplicates(inplace=True)

In [17]:
print("Jumlah duplikasi: ", dataset1_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [18]:
dataset1_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,15
link,23675


In [19]:
dataset1_df['functions'] = dataset1_df['functions'].fillna("Unknown")

In [20]:
dataset1_df['link'] = dataset1_df['link'].fillna("No Link")

In [21]:
print(dataset1_df.isna().sum())


ingredient_name    0
rating             0
functions          0
link               0
dtype: int64


#### Cleaning Tabel dataset2_df

##### Duplicate Data

In [22]:
dataset2_df.duplicated().sum()

0

In [23]:
dataset2_df.drop_duplicates(inplace=True)

In [24]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [25]:
dataset2_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,14
link,23675
benefits,24550
categories,23724
glance,25132
references,25194
all,23692
description,23704


In [26]:
dataset2_df['functions'] = dataset2_df['functions'].fillna("Unknown")

In [27]:
print(dataset2_df.isna().sum())

ingredient_name        0
rating                 0
functions              0
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64


### Drop Column Link

In [28]:
dataset1_df = dataset1_df.drop(columns='link')

In [29]:
dataset2_df = dataset2_df.drop(columns=['benefits', 'categories', 'glance', 'references', 'all', 'description', 'combined_text', 'link'])

### Exploratory Data Analysis (EDA)

#### Explore Tabel dataset1_df

In [30]:
dataset1_df.sample(5)

,ingredient_name,rating,functions
23699,Sr-Oligopeptide-150 Silkworm Polypeptide-2 Dip...,NOT RATED,Functions: Skin Conditioning. We have not yet ...
16041,Nymphaea Stellata Flower Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
7953,Cosmos Bipinnatus Phytoplacenta Extract,NOT RATED,"Functions: Antimicrobial, Antioxidant, Hair Co..."
11035,Glyceryl Tetrahydrofarnesylacetate,NOT RATED,"Functions: Cleansing Agent, Emulsifier. We hav..."
23853,Stearyl Behenate,NOT RATED,"Functions: Skin Conditioning, Emollient. We ha..."


In [31]:
dataset1_df.describe(include="all")

,ingredient_name,rating,functions
count,26087,26087,26087
unique,26084,6,4056
top,Albumen,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,2,23675,5642


In [32]:
dataset1_df.ingredient_name.is_unique

False

In [33]:
dataset1_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26082                        Zostera Marina Extract
26083       Zygophyllum Qatarense Leaf/Stem Extract
26084    Zygosaccharomyces Microellipsoides Ferment
26085                     Zymomonas Ferment Extract
26086                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26087, dtype: object>

In [34]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset1_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)


                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26079                          Zymomonas Ferment Extract   
26080                         Zymomonas Ferment Filtrate   
26081                                biosaccharide gum-1   
26082                                        pH Adjuster   
26083                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [35]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset1_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)


                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4056 rows x 1 columns]


#### Explore Tabel dataset2_df

In [36]:
dataset2_df.sample(5)

,ingredient_name,rating,functions
18120,Pentaerythrityl Tetraethylhexanoate/Benzoate,NOT RATED,"Functions: Hair Conditioning, Emollient. We ha..."
9928,Epoxycyclododecane,NOT RATED,Functions: Fragrance. We have not yet rated th...
10248,Ethyl Perfluorobutyl Ether,NOT RATED,Functions: Solvent. We have not yet rated this...
16704,Osmanthus Fragrans Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
17501,PEG-2 Coco-Benzonium Chloride,NOT RATED,"Functions: Antistatic, Cleansing Agent. We hav..."


In [37]:
dataset2_df.describe(include="all")

,ingredient_name,rating,functions
count,26074,26074,26074
unique,26074,6,4045
top,3-O Ethyl Ascorbic Acid,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,1,23675,5642


In [38]:
dataset2_df.ingredient_name.is_unique

True

In [39]:
dataset2_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26069                        Zostera Marina Extract
26070       Zygophyllum Qatarense Leaf/Stem Extract
26071    Zygosaccharomyces Microellipsoides Ferment
26072                     Zymomonas Ferment Extract
26073                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26074, dtype: object>

In [40]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset2_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)

                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26069                          Zymomonas Ferment Extract   
26070                         Zymomonas Ferment Filtrate   
26071                                biosaccharide gum-1   
26072                                        pH Adjuster   
26073                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [41]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset2_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)

                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4045 rows x 1 columns]


##### Menggabungkan dataset1 dan dataset2

In [42]:
datasetAll_df = pd.concat([dataset1_df, dataset2_df], axis=0, ignore_index=True)
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


### Export Clean Dataset

In [43]:
# Menyimpan dataset ke file CSV
datasetAll_df.to_csv("/content/sample_data/datasetAll.csv", index=False)


In [44]:
dataset1_df.to_csv("/content/sample_data/dataset1_clean.csv", index=False)
dataset2_df.to_csv("/content/sample_data/dataset2_clean.csv", index=False)

### Pre-processing

In [45]:
# import data
datasetAll_df = pd.read_csv('/content/sample_data/datasetAll.csv')
datasetAll_df #show data below

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


In [46]:
print(datasetAll_df['rating'].unique())

['BEST' 'GOOD' 'AVERAGE' 'BAD' 'WORST' 'NOT RATED']


In [47]:
# Mengonversi nilai non-numerik ke angka
rating_mapping = {
    'BEST': 6,
    'GOOD': 5,
    'AVERAGE': 4,
    'BAD': 3,
    'WORST':2,
    'NOT RATED':1
}

datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)

<ipython-input-47-6eb023509551>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)


In [48]:
# Hapus baris yang tidak sesuai dengan konteks
datasetAll_df = datasetAll_df[datasetAll_df['rating'].apply(lambda x: str(x).isnumeric() or x in rating_mapping.keys())]


In [49]:
datasetAll_df['rating'] = pd.to_numeric(datasetAll_df['rating'], errors='coerce')

In [50]:
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,6,This potent antioxidant is a highly stable for...
1,Acai,6,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,6,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,6,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,6,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,1,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,1,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,1,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,1,"Functions: Humectant, Skin Conditioning. We ha..."


In [51]:
print(datasetAll_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52161 entries, 0 to 52160
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  52161 non-null  object
 1   rating           52161 non-null  int64 
 2   functions        52161 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
None


## DOWNLOAD OCR

In [54]:
!apt-get update
!apt-get install -y tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64

In [55]:
!pip install pytesseract

In [56]:
!tesseract --version

tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


### Prepare the dataset for training

In [150]:
vocab_size = 15000
embedding_dim = 64
max_length = 200
trunc_type='post'
padding_type='post'
oov_token = "<OOV>"
training_portion = 0.8

In [151]:
# Dataset Preparation
# Assume `datasetAll_df` is a pandas DataFrame with columns: ingredient_name, rating, functions
ingredient_texts = datasetAll_df['ingredient_name']
ratings = datasetAll_df['rating']
functions = datasetAll_df['functions']

In [152]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding for Ratings
le = LabelEncoder()
ratings_encoded = le.fit_transform(ratings)

In [153]:
# Tokenizer for ingredient_name
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(ingredient_texts)

# Sequences and Padding
ingredient_sequences = tokenizer.texts_to_sequences(ingredient_texts)
ingredient_padded = pad_sequences(ingredient_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    ingredient_padded, ratings_encoded, test_size=0.2, random_state=42
)

print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

Training Data Shape: (41728, 200)
Test Data Shape: (10433, 200)


In [154]:
#Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')  # Softmax for multiclass classification
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [155]:
# Model Training
history = model.fit(
    X_train, y_train,
    epochs=10,  # Adjust epochs based on needs
    validation_data=(X_test, y_test),
    batch_size=32,
    verbose=1
)

Epoch 1/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9059 - loss: 0.4690 - val_accuracy: 0.9076 - val_loss: 0.4256
Epoch 2/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.9080 - loss: 0.4246 - val_accuracy: 0.9076 - val_loss: 0.4243
Epoch 3/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.9099 - loss: 0.4169 - val_accuracy: 0.9076 - val_loss: 0.4262
Epoch 4/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.9068 - loss: 0.4271 - val_accuracy: 0.9076 - val_loss: 0.4211
Epoch 5/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 16s 12ms/step - accuracy: 0.9078 - loss: 0.4212 - val_accuracy: 0.9076 - val_loss: 0.4158
Epoch 6/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9082 - loss: 0.4159 - val_accuracy: 0.9076 - val_loss: 0.4103
Epoch 7/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.9094 - loss: 0.4022 - val_accuracy: 0.9076 - val_loss: 0.3955
Epoch 8/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.9073 -

In [156]:
# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy:.2f}")

327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9079 - loss: 0.3664

Test Accuracy: 0.91


In [157]:
import pytesseract
from PIL import Image

# Load image
image = Image.open('/content/image_training/skintific.jpg')  # Ganti dengan path gambar yang sesuai

# Gunakan pytesseract untuk ekstrak teks dari gambar
detected_text = pytesseract.image_to_string(image)

print(f"Detected Text: {detected_text}")

Detected Text: SKINTIFIC

MUGWORT

ANT\ PORES & AC

Salicylic Acid, Centella — Je

NE CLAY MASK

Niacinamide,

BI Unclogs & Refines Pores

Bl Prevents & Reduces Breakouts

55g / 1.83 FL.OZ

 



In [158]:
detected_text = pytesseract.image_to_string(image, config="--psm 6")

In [159]:
detected_tokens = detected_text.lower().split()

In [160]:
ingredient_sequences = tokenizer.texts_to_sequences([detected_text])
matched_sequences = [seq for seq in ingredient_sequences if seq != []]

if matched_sequences:
    print("Matched Sequences:", matched_sequences)
else:
    print("No matching sequences found.")


Matched Sequences: [[1, 5894, 6522, 1, 1, 692, 1, 683, 3649, 15, 640, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 17, 1410, 1, 1, 1]]


In [161]:
word_index = tokenizer.word_index
reversed_word_index = {v: k for k, v in word_index.items()}

matched_words = [reversed_word_index[idx] for seq in matched_sequences for idx in seq]
print("Matched Words:", matched_words)

Matched Words: ['<OOV>', 'mugwort', 'ant', '<OOV>', '<OOV>', 'clay', '<OOV>', 'niacinamide', 'salicylic', 'acid', 'centella', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '<OOV>', '1', '83', '<OOV>', '<OOV>', '<OOV>']


In [162]:
if matched_words:
    input_sequence = tokenizer.texts_to_sequences([" ".join(matched_words)])
    input_padded = pad_sequences(input_sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    predicted_index = model.predict(input_padded).argmax(axis=1)[0]
    predicted_rating = le.inverse_transform([predicted_index])[0]
    predicted_function = functions[predicted_index]

    print(f"Rating: {predicted_rating}")
    print(f"Function: {predicted_function}")
else:
    print("No valid ingredient detected for prediction.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Rating: 1
Function: This potent antioxidant is a highly stable form of vitamin C that improves skin tone, helps fade discolorations, and visibly firms skin.


In [163]:
# Muat model
from tensorflow.keras.models import load_model
loaded_model = load_model("ingredient_rating_model.h5")

# Kompilasi ulang model
loaded_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Evaluasi model (contoh)
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9079 - loss: 0.3860
Test Accuracy: 0.91


In [164]:
# Save model and tokenizer
model.save("ingredient_function_model.h5")

In [170]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Load tokenizer dan model untuk prediksi
with open('tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)

loaded_model = tf.keras.models.load_model("ingredient_function_model.h5")

In [173]:
import tensorflow as tf

# Muat model TensorFlow
loaded_model = tf.keras.models.load_model("ingredient_function_model.h5")

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Simpan model TFLite ke file
with open("ingredient_function_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model berhasil dikonversi ke TFLite.")


Saved artifact at '/tmp/tmpqvk0doir'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(32, 200), dtype=tf.float32, name='input_layer_7')
Output Type:
  TensorSpec(shape=(32, 6), dtype=tf.float32, name=None)
Captures:
  137160973517744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137160971773984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137160971776800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137160972840704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137160972841760: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model berhasil dikonversi ke TFLite.


In [174]:
from flask import Flask, request, jsonify
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load TFLite Model
interpreter = tf.lite.Interpreter(model_path="ingredient_function_model.tflite")
interpreter.allocate_tensors()

# Load tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = joblib.load(f)

# Hyperparameters (sesuai model)
max_length = 200
padding_type = 'post'
trunc_type = 'post'

# Setup Flask
app = Flask(__name__)

# Define helper function for prediction
def predict_skincare_tflite(text):
    # Tokenize and pad the text
    input_sequence = tokenizer.texts_to_sequences([text])
    input_padded = pad_sequences(input_sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # Set input tensor
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_padded.astype(np.float32))

    # Run inference
    interpreter.invoke()

    # Get prediction
    prediction = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    predicted_index = np.argmax(prediction)

    # Define labels and recommendations
    functions = ["Moisturizing", "Brightening", "Anti-Aging", "Soothing"]  # Contoh fungsi
    predicted_function = functions[predicted_index]
    predicted_rating = prediction[0][predicted_index]

    # Recommendation logic (ubah sesuai kebutuhan)
    recommendation = "Safe for daily use" if predicted_rating > 0.7 else "Use with caution"

    return {"predicted_function": predicted_function, "predicted_rating": predicted_rating, "recommendation": recommendation}

# Define endpoint for OCR and prediction
@app.route('/scan', methods=['POST'])
def scan():
    try:
        # Get image file from request
        image_file = request.files.get('image')
        if not image_file:
            return jsonify({"error": "No image file provided"}), 400

        # Open image and apply OCR
        image = Image.open(image_file)
        detected_text = pytesseract.image_to_string(image, config="--psm 6")

        # Clean text and predict
        detected_text_cleaned = detected_text.lower().strip()
        result = predict_skincare_tflite(detected_text_cleaned)

        # Return result
        return jsonify({"detected_text": detected_text_cleaned, **result})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Run Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [175]:
# Prediksi dari teks input
detected_ingredient = "3-O Ethyl Ascorbic Acid"  # Contoh input
input_sequence = loaded_tokenizer.texts_to_sequences([detected_ingredient])
input_padded = pad_sequences(input_sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

predicted_index = loaded_model.predict(input_padded).argmax(axis=1)[0]
predicted_rating = le.inverse_transform([predicted_index])[0]

# Cari fungsi berdasarkan teks ingredient_name di dataset asli
matching_row = datasetAll_df[datasetAll_df['ingredient_name'].str.lower() == detected_ingredient.lower()]
if not matching_row.empty:
    predicted_function = matching_row['functions'].values[0]
else:
    predicted_function = "Unknown function"

print(f"Detected Ingredient: {detected_ingredient}")
print(f"Predicted Rating: {predicted_rating}")
print(f"Function: {predicted_function}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Detected Ingredient: 3-O Ethyl Ascorbic Acid
Predicted Rating: 1
Function: This potent antioxidant is a highly stable form of vitamin C that improves skin tone, helps fade discolorations, and visibly firms skin.
